In [1]:
# !pip install umap-learn -i https://mirrors.ustc.edu.cn/pypi/web/simple --user
# conda install -c conda-forge hdbscan

In [2]:
import re
import hdbscan
import umap
from sentence_transformers import SentenceTransformer

# for LDA topic modelling
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools

In [3]:
key_yr_chap_mp = {'Karamchand_Gandhi_1869': ['1\\chapter1'],
             'Alias_Kaba_Gandhi_1869': ['1\\chapter1'],
             'Uttamchand_Gandhi_1869': ['1\\chapter1'],
             'Faithfulness_1869': ['1\\chapter4'],
             'Smoking_Stumps_1887': ['1\\chapter7'],
             'Cigarettes_1887': ['1\\chapter7'],
             'Cigarette_1887': ['1\\chapter7'],
             'Vaishnava_Faith_1887': ['1\\chapter9'],
             'Ahmedabad_1887': ['1\\chapter10'],
             'Ahmedabad_1915': ['5\\chapter9', '5\\chapter20'],
             'Matriculation_Examination_1887': ['1\\chapter10'],
             'Kathiawad_Students_1887': ['1\\chapter10'],
             'Ringworm_1887': ['1\\chapter13'],
             'Vegetarianism_1887': ['1\\chapter14'],
             'Vegetarianism_1890': ['1\\chapter16'],
             'Vegetarianism_1903': ['4\\chapter6'],
             'Dietary_1887': ['1\\chapter14'],
             'Diet_1887': ['1\\chapter14'],
             'Vegetarian_1890': ['1\\chapter16'],
             'Narayan_Hemchandra_1890': ['1\\chapter21'],
             'Time_Narayan_Hemchandra_1890': ['1\\chapter21'],
             'Narayan_1890': ['1\\chapter21'],
             'Eiffel_Tower_1890': ['1\\chapter22'],
             'Legal_Maxims_1857': ['1\\chapter24'],
             'Briefless_Barristers_1893': ['2\\chapter3'],
             'Barrister_1893': ['2\\chapter3', '2\\chapter4'],
             'Barrister_1885': ['2\\chapter14'],
             'Barrister_1890': ['4\\chapter46'],
             'Barristers_1893': ['2\\chapter3'],
             'Bombay_Commissions_1893': ['2\\chapter4'],
             'Port_Natal_1893': ['2\\chapter7'],
             'Christian_Indians_1924': ['2\\chapter8'],
             'Prayers_1893': ['2\\chapter11'],
             'Prayer_1893': ['2\\chapter11'],
             'Prayer_1885': ['2\\chapter15'],
             'Transvaal_1885': ['2\\chapter13'],
             'Transvaal_1920': ['3\\chapter4'],
             'Transvaal_1903': ['4\\chapter2'],
             'Transvaal_1890': ['4\\chapter46'],
             'Litigation_1885': ['2\\chapter14'],
             'Natal_Legislative_Assembly_1893': ['2\\chapter16'],
             'Durban_1893': ['2\\chapter16'],
             'Sheth_Haji_Muhammad_1893': ['2\\chapter17'],
             'Sheth_Haji_Muhammad_Haji_Dada_1893': ['2\\chapter17'],
             'Financially_Sheth_Abdulla_Haji_1893': ['2\\chapter17'],
             'Bombay_High_Court_1893': ['2\\chapter18'],
             'Court_1893': ['2\\chapter18'],
             'Natal_Indian_Congress_1894': ['2\\chapter19', '2\\chapter20'],
             'Natal_Indians_1860': ['2\\chapter21'],
             'Natal_Zulus_1860': ['2\\chapter21'],
             'Taxation_1860': ['2\\chapter21'],
             'Servant_1896': ['2\\chapter23'],
             'Household_1896': ['2\\chapter23'],
             'Loyalty_1896': ['2\\chapter26'],
             'Sir_Pherozeshah_1896': ['2\\chapter28'],
             'Calcutta_1896': ['2\\chapter29'],
             'Raja_Sir_Pyarimohan_Mukarji_1896': ['2\\chapter29'],
             'Bengal_Club_1896': ['2\\chapter29'],
             'Hindu_Wife_1920': ['3\\chapter1'],
             'Hindu_Husband_1920': ['3\\chapter1'],
             'Quarantine_1920': ['3\\chapter2'],
             'Quarantine_Officer_1920': ['3\\chapter2'],
             'Plague_1920': ['3\\chapter2'],
             'Brahmacharya_Vow_1906': ['3\\chapter7', '3\\chapter8'],
             'Brahmacharya_1906': ['3\\chapter7'],
             'Brahmacharya_1912': ['4\\chapter30'],
             'Vow_1906': ['3\\chapter8'],
             'Washing_Outfit_1906': ['3\\chapter9'],
             'Laundry_1906': ['3\\chapter9'],
             'Washing_1906': ['3\\chapter9'],
             'Boer_War_1899': ['3\\chapter10'],
             'Boer_War_1914': ['4\\chapter39'],
             'Sir_Pherozeshah_Mehta_1901': ['3\\chapter13'],
             'Babu_Bhupendranath_Basu_1901': ['3\\chapter14'],
             'India_Club_1901': ['3\\chapter16'],
             'Professor_Ray_1896': ['3\\chapter17'],
             'Gokhlae_1896': ['3\\chapter18'],
             'Christian_Indian_1896': ['3\\chapter18'],
             'Babu_Kalicharan_Banerji_1896': ['3\\chapter18'],
             'Brahmo_Samaj_1896': ['3\\chapter19'],
             'Adi_Brahmo_Samaj_1896': ['3\\chapter19'],
             'Maharshi_Devendranath_1896': ['3\\chapter19'],
             'Indian_Evidence_Act_1919': ['3\\chapter21'],
             'Insult_1903': ['4\\chapter3'],
             'Hindu_Influence_1903': ['4\\chapter5'],
             'Hinduism_1903': ['4\\chapter5'],
             'Hindu_1903': ['4\\chapter5'],
             'Laxative_Medicines_1903': ['4\\chapter7'],
             'Laxatives_1903': ['4\\chapter7'],
             'Indian_Opinion_1900': ['4\\chapter8'],
             'Indian_Opinion_1904': ['4\\chapter19'],
             'Hindus_1904': ['4\\chapter14'],
             'Zulus_1900': ['4\\chapter25'],
             'Zulu_1900': ['4\\chapter25'],
             'Satyagraha_1900': ['4\\chapter26'],
             'Satyagraha_1919': ['5\\chapter24'],
             'Word_Sadagraha_1900': ['4\\chapter26'],
             'Result_Maganlal_Gandhi_1900': ['4\\chapter26'],
             'Curry_Powder_1914': ['4\\chapter29'],
             'Fasting_1912': ['4\\chapter31'],
             'Tolstoy_Farm_1911': ['4\\chapter35'],
             'Commanding_Officer_1890': ['4\\chapter40'],
             'Commanding_1890': ['4\\chapter40'],
             'Parsi_Rustomji_1890': ['4\\chapter47'],
             'Bombay_Gokhale_1915': ['5\\chapter2'],
             'Kakasaheb_1915': ['5\\chapter4'],
             'Kalelkar_1915': ['5\\chapter4'],
             'Teachers_Family_Names_1915': ['5\\chapter4'],
             'Acharya_Ramadevji_1915': ['5\\chapter8'],
             'Gurukul_1915': ['5\\chapter8'],
             'Gujarat_1915': ['5\\chapter9'],
             'Satyagraha_Ashram_1915': ['5\\chapter9'],
             'Amritlal_Thakkar_1915': ['5\\chapter10'],
             'Ashram_1915': ['5\\chapter10', '5\\chapter21'],
             'Abolition_1894': ['5\\chapter11'],
             'Indigo_1917': ['5\\chapter12'],
             'Professor_Kripalani_1915': ['5\\chapter13'],
             'Maulana_Mazharul_Haq_1915': ['5\\chapter13'],
             'Champaran_Inquiry_1915': ['5\\chapter16'],
             'Champaran_Satyagraha_1915': ['5\\chapter16'],
             'Rajendrababu_1915': ['5\\chapter17'],
             'Janakdharibabu_1915': ['5\\chapter17'],
             'Gangadharrao_Deshpande_1915': ['5\\chapter17'],
             'Sanitation_Work_1915': ['5\\chapter18'],
             'Sanitation_1915': ['5\\chapter18'],
             'Treatment_1915': ['5\\chapter18'],
             'Bihar_Government_1915': ['5\\chapter19'],
             'Ashram_Walls_1915': ['5\\chapter21'],
             'Ashram_Children_1915': ['5\\chapter21'],
             'Kheda_Satyagraha_1919': ['5\\chapter23'],
             'Mohanlal_Pandya_1919': ['5\\chapter23'],
             'Shankarlal_Parikh_1919': ['5\\chapter23'],
             'Satyagrahi_Volunteers_1919': ['5\\chapter24'],
             'Satyagrahi_1919': ['5\\chapter24'],
             'Mamlatdar_1919': ['5\\chapter25'],
             'Hartal_1919': ['5\\chapter31'],
             'Swami_Shraddhanandji_1919': ['5\\chapter31'],
             'Meanwhile_Delhi_1919': ['5\\chapter31'],
             'Kheda_Satyagraha_Struggle_1919': ['5\\chapter33'],
             'Jalianwala_Bagh_Tragedy_1919': ['5\\chapter34'],
             'Punjab_Situation_1919': ['5\\chapter34'],
             'Punjab_Government_1917': ['5\\chapter35', '5\\chapter37'],
             'Jalianwala_Bagh_Massacre_1917': ['5\\chapter35', '5\\chapter38'],
             'Punjab_Leaders_1917': ['5\\chapter35'],
             'Pandit_Motilal_Nehru_1917': ['5\\chapter37'],
             'Pandit_1917': ['5\\chapter37'],
             'Slivers_1917': ['5\\chapter40'],
             'Swadeshi_Agitation_1920': ['5\\chapter41'],
             'Swadeshi_Movement_1920': ['5\\chapter41'],
             'Khilafat_Agitation_1920': ['5\\chapter42']}

In [4]:
filtered_keywords = ["Vegetarianism_1890","Satyagraha_1900","Jalianwala_Bagh_Massacre_1917","Barrister_1893","Sir_Pherozeshah_Mehta_1901"]

In [5]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def create_representative_doc(corpus,query):
    paragraph = []
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    print(f"length of the texts {len(corpus)}")
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     paragraph = ""
    for i,score in enumerate(cos_scores):
        if score >0.25:
            paragraph.append((i,corpus[i]))
    for i in range(len(paragraph)-1):
        idx,para = paragraph[i][0],paragraph[i][1]
        if paragraph[i+1][0] - idx ==2:
            paragraph.append((idx+1,corpus[idx+1]))
#         elif paragraph[i+1][0] - idx >4:
#             paragraph.append((idx+0.5," \n "))
        
    paragraph = sorted(paragraph, key = lambda x: x[0])
    res = []
    for idx,para in paragraph:
        res.append(para)
#         res += (para+'. ')     
    return res


In [6]:
def split_sentence_from_text(key):
    file_loc = r"C:\Users\hp\OneDrive\Documents\topic_coherence\historical_books\SOMEWT\part%s.txt"%key
    text = ""
    with open(file_loc) as f:
        text = f.read()
        text = re.sub(r"I ", "Mahatma Gandhi ",text)
        text = re.sub(r" me ", " Mahatma Gandhi ",text)
        text = re.sub(r" my ", " Mahatma Gandhi's ",text)
        text = re.sub(r" My ", " Mahatma Gandhi's ",text)
        text = re.sub(r" myself ", " Mahatma Gandhi ",text)
        text = re.sub(r" me,", "  Mahatma Gandhi,",text)
        text = re.sub(r"we ","they " ,text)
        text = re.sub(r"We ","They " ,text)
        #

    res = re.findall(r"[^.!?]+", text)
    
    return res

# key_yr_chap_mp

mp_key_doc_2 = {}
for key in filtered_keywords:
#         mp_key_doc_2[key] = ""
        print(f"{key}  -> {key_yr_chap_mp[key]}")
        for chap in key_yr_chap_mp[key]:
            corpus = split_sentence_from_text(chap)
            mp_key_doc_2[key] = create_representative_doc(corpus,key)

# print(mp_key_doc_2)

Vegetarianism_1890  -> ['1\\chapter16']
length of the texts 76
Satyagraha_1900  -> ['4\\chapter26']
length of the texts 19
Jalianwala_Bagh_Massacre_1917  -> ['5\\chapter35', '5\\chapter38']
length of the texts 61
length of the texts 50
Barrister_1893  -> ['2\\chapter3', '2\\chapter4']
length of the texts 94
length of the texts 80
Sir_Pherozeshah_Mehta_1901  -> ['3\\chapter13']
length of the texts 84


In [7]:
corpus = mp_key_doc_2['Vegetarianism_1890']

### Wikipedia section title generation

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-small-medium-title-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-small-medium-title-generation")


def get_title(text):
    inputs = ["summarize: " + text]
    inputs = tokenizer(inputs, max_length=768, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
    return predicted_title

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### KMeans Clustering

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  2
[' Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects', " Ethically they had arrived at the  conclusion that man's supremacy over the lower animals meant not that  the former should prey upon the latter, but that the higher should  protect the lower, and that there should be mutual aid between the two  as between man and man", '  According to the practical and economic argument they had demonstrated  that a vegetarian diet was the least expensive', ' There was a Vegetarian Society in England  with a weekly journal of its own', ' Oldfield who was Editor of  the The Vegetarian became President']

Cluster  4
[' They had also brought out the truth that man  eats not for enjoyment but to live', ' And some of them accordingly  suggested and effected in their lives abstention not only from  flesh-meat but from eggs and milk', " Scientifically some had concluded  tha

In [19]:
mp = {}
for k,v in clustered_sentences.items():
    mp[k] = '.'.join(v)
    mp[k]+='.'
    
mp2 = {}
for k,v in mp.items():
    mp2[k] = get_title(v)
    
sec_to_para = {}
for k,v in mp2.items():
    sec_to_para[v] = mp[k]
for k,v in sec_to_para.items():
    print(f"{k}  ->   {v}")
    print()
    print()

The Vegetarian Society in England: The Vegetarian Society  ->    Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects. Ethically they had arrived at the  conclusion that man's supremacy over the lower animals meant not that  the former should prey upon the latter, but that the higher should  protect the lower, and that there should be mutual aid between the two  as between man and man.  According to the practical and economic argument they had demonstrated  that a vegetarian diet was the least expensive. There was a Vegetarian Society in England  with a weekly journal of its own. Oldfield who was Editor of  the The Vegetarian became President.


Man eats not for enjoyment but to live.  ->    They had also brought out the truth that man  eats not for enjoyment but to live. And some of them accordingly  suggested and effected in their lives abstention not only from  flesh-m

### Aglomerative clustering

In [16]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering

corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  4
[' Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects', '  According to the practical and economic argument they had demonstrated  that a vegetarian diet was the least expensive', ' There was a Vegetarian Society in England  with a weekly journal of its own', ' Oldfield who was Editor of  the The Vegetarian became President']

Cluster  1
[" Ethically they had arrived at the  conclusion that man's supremacy over the lower animals meant not that  the former should prey upon the latter, but that the higher should  protect the lower, and that there should be mutual aid between the two  as between man and man", ' They had also brought out the truth that man  eats not for enjoyment but to live', ' And some of them accordingly  suggested and effected in their lives abstention not only from  flesh-meat but from eggs and milk', " Scientifically some had concluded  tha

In [17]:
mp = {}
for k,v in clustered_sentences.items():
    mp[k] = '.'.join(v)
    mp[k]+='.'
    
mp2 = {}
for k,v in mp.items():
    mp2[k] = get_title(v)
    
sec_to_para = {}
for k,v in mp2.items():
    sec_to_para[v] = mp[k]
for k,v in sec_to_para.items():
    print(f"{k}  ->   {v}")
    print()
    print()

Mahatma Gandhi saw that the writers on vegetarianism had examined the question very minutely, attacking it in its religious, scientific, practical and medical aspects  ->    Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects.  According to the practical and economic argument they had demonstrated  that a vegetarian diet was the least expensive. There was a Vegetarian Society in England  with a weekly journal of its own. Oldfield who was Editor of  the The Vegetarian became President.


Man's Supremeacy over the lower animals meant not that the latter should prey upon the latter, but that there should be mutual aid between the two as between man and man  ->    Ethically they had arrived at the  conclusion that man's supremacy over the lower animals meant not that  the former should prey upon the latter, but that the higher should  protect the lower, and that there should

### Inspired by Bertopic

In [20]:
# Load sentence transformer model
sentence_model = SentenceTransformer("paraphrase-distilroberta-base-v1")

# Create documents embeddings
embeddings = sentence_model.encode(corpus, show_progress_bar=False)

# Define UMAP model to reduce embeddings dimension
umap_model = umap.UMAP(n_neighbors=15,
                       n_components=2,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False)

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

hdbscan_model.fit(embeddings)
cluster_assignment = hdbscan_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  2
[' Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects', ' All these considerations  had their effect on  Mahatma Gandhi, and Mahatma Gandhi came across vegetarians of all these types  in vegetarian restaurants', ' Mahatma Gandhi subscribed to the weekly, joined the  society and very shortly found Mahatma Gandhi on the Executive Committee', " Here Mahatma Gandhi  came in contact with those who were regarded as pillars of  vegetarianism, and began Mahatma Gandhi's own experiments in dietetics", ' Vegetarianism was then a new cult in England,  and likewise for  Mahatma Gandhi, because, as they have seen, Mahatma Gandhi had gone there a  convinced meat-eater, and was intellectually converted to vegetarianism  later', " Full of the neophyte's zeal for vegetarianism, Mahatma Gandhi decided to start  a vegetarian club in Mahatma Gandhi's locality, Bayswater"]

Clust

In [21]:
mp = {}
for k,v in clustered_sentences.items():
    mp[k] = '.'.join(v)
    mp[k]+='.'
    
mp2 = {}
for k,v in mp.items():
    mp2[k] = get_title(v)
    
sec_to_para = {}
for k,v in mp2.items():
    sec_to_para[v] = mp[k]
for k,v in sec_to_para.items():
    print(f"{k}  ->   {v}")
    print()
    print()

Mahatma Gandhi started a vegetarian club in Bayswater, Mahatma Gandhi's locality  ->    Mahatma Gandhi saw that the writers on vegetarianism had examined the  question very minutely, attacking it in its religious, scientific,  practical and medical aspects. All these considerations  had their effect on  Mahatma Gandhi, and Mahatma Gandhi came across vegetarians of all these types  in vegetarian restaurants. Mahatma Gandhi subscribed to the weekly, joined the  society and very shortly found Mahatma Gandhi on the Executive Committee. Here Mahatma Gandhi  came in contact with those who were regarded as pillars of  vegetarianism, and began Mahatma Gandhi's own experiments in dietetics. Vegetarianism was then a new cult in England,  and likewise for  Mahatma Gandhi, because, as they have seen, Mahatma Gandhi had gone there a  convinced meat-eater, and was intellectually converted to vegetarianism  later. Full of the neophyte's zeal for vegetarianism, Mahatma Gandhi decided to start  a veget

## LDA Topic Modelling using gensim

In [24]:
data = [' All these considerations  had their effect on  Mahatma Gandhi, and Mahatma Gandhi came across vegetarians of all these types  in vegetarian restaurants', ' Mahatma Gandhi subscribed to the weekly, joined the  society and very shortly found Mahatma Gandhi on the Executive Committee', " Here Mahatma Gandhi  came in contact with those who were regarded as pillars of  vegetarianism, and began Mahatma Gandhi's own experiments in dietetics", ' Mahatma Gandhi knew that her definition of  meat included eggs', ' Mahatma Gandhi came across  three definitions of meat in England', " Mahatma Gandhi's mother's interpretation of meat  was, according to the golden rule, the only true one for  Mahatma Gandhi, and not the  one Mahatma Gandhi's wider experience or Mahatma Gandhi's pride of better knowledge might have  taught me", " Mahatma Gandhi's experiments in England were conducted from the point of view of economy  and hygiene", ' Vegetarianism was then a new cult in England,  and likewise for  Mahatma Gandhi, because, as they have seen, Mahatma Gandhi had gone there a  convinced meat-eater, and was intellectually converted to vegetarianism  later', " Full of the neophyte's zeal for vegetarianism, Mahatma Gandhi decided to start  a vegetarian club in Mahatma Gandhi's locality, Bayswater"]


In [25]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['all', 'these', 'considerations', 'had', 'their', 'effect', 'on', 'mahatma', 'gandhi', 'and', 'mahatma', 'gandhi', 'came', 'across', 'vegetarians', 'of', 'all', 'these', 'types', 'in', 'vegetarian', 'restaurants']]


In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['all', 'these', 'considerations', 'had', 'their', 'effect', 'on', 'mahatma', 'gandhi', 'and', 'mahatma', 'gandhi', 'came', 'across', 'vegetarians', 'of', 'all', 'these', 'types', 'in', 'vegetarian', 'restaurants']


In [28]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['consideration', 'effect', 'come', 'type', 'vegetarian', 'restaurant']]


In [31]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [32]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [33]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.043*"meat" + 0.034*"come" + 0.034*"vegetarianism" + 0.026*"definition" + '
  '0.017*"economy" + 0.017*"point" + 0.017*"hygiene" + 0.017*"view" + '
  '0.017*"taught" + 0.017*"well"')]
